In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
from denn import *
import pandas as pd
import io #Used as buffer

In [2]:
path= Path(f'../../data/cluster_results')

In [3]:
def read_csv(f):
    df = pd.read_csv(f).mean()
    df['std'] = pd.read_csv(f).std().iloc[0] ##added line for std, remove this will remove std from table
    df['experiment'] = f.parent.parent.parent.name
    df['function'] = f.parent.parent.name
    name = f.parent.name
    #print(name)
    df['freq'] = float(name[4:].split('n')[0])
    #print(df['freq'])
    df['method'] = f.name.split('_')[0]+'_'+f.name.split('_')[1]
    #df['method'] = f.name.split('_')[0]
    #print(df['method'])
    return df

measure_name=['mof','sr','arr','nfe','nn_time']
index_measure=0

In [16]:
m = measure_name[index_measure]
files  = list(f for f in path.glob(f'exp*/*/freq1*nn_p1nn_s2nn_tw5*bs4*/*{m}.csv') if 'Worst' in f.name)
# files += list(path.glob(f'exp*/*/*nn_p3/*{m}.csv'))
files += [f for f in path.glob(f'exp*/*/freq1*nn_s2nn_tw5*bs4*/*{m}.csv') if 'noNN' in f.name] #f'exp*/*/*nn_p3/*{m}.csv'
#print(files)
df = pd.DataFrame([read_csv(f) for f in files])
# data = pd.pivot_table(df, index=['experiment','function'], columns='method', values=m) #,'freq' later for adding frequency experiment
data = pd.pivot_table(df, index=['experiment','function','freq'],  columns='method') #,'freq' later for adding frequency experiment, ,values=m
data.style.format("{:,.2f}")

In [18]:
#to add std to table
final_data = data[m].copy()

for col in data[m].columns:
    final_data[col] = data[m][col].apply(lambda x: f'{x:,.2f} (±') + data['std'][col].apply(lambda x: f'{x:,.2f})')
final_data

method                                       NNdrop_Worst  \
experiment function   freq                                  
exp1       rastrigin  1.0                   16.37 (±7.50)   
           rosenbrock 1.0        144,398.14 (±185,487.12)   
           sphere     1.0                   10.03 (±6.74)   
exp2       rastrigin  1.0                    2.01 (±0.25)   
           rosenbrock 1.0                 180.25 (±63.82)   
           sphere     1.0                    0.26 (±0.17)   
exp3       rastrigin  1.0                   13.36 (±3.07)   
           rosenbrock 1.0              4,120.18 (±134.21)   
           sphere     1.0                    4.82 (±1.39)   
exp4       rastrigin  1.0                   67.35 (±3.59)   
           rosenbrock 1.0   14,769,086.73 (±1,579,226.67)   
           sphere     1.0                   57.78 (±3.72)   

method                                       NNnorm_Worst  \
experiment function   freq                                  
exp1       rastrigin  1.0                   16.60 (±6.60)   
           rosenbrock 1.0        126,306.82 (±129,777.91)   
           sphere     1.0                    6.45 (±6.59)   
exp2       rastrigin  1.0                    1.96 (±0.24)   
           rosenbrock 1.0                  85.58 (±20.33)   
           sphere     1.0                    0.12 (±0.01)   
exp3       rastrigin  1.0                   15.65 (±3.67)   
           rosenbrock 1.0            2,051.71 (±1,330.53)   
           sphere     1.0                    3.90 (±1.29)   
exp4       rastrigin  1.0                   67.50 (±5.56)   
           rosenbrock 1.0   10,297,506.65 (±1,621,286.07)   
           sphere     1.0                   43.83 (±3.40)   

method                                  noNNReval_mof.csv  
experiment function   freq                                 
exp1       rastrigin  1.0                   21.90 (±0.47)  
           rosenbrock 1.0           83,986.73 (±3,445.84)  
           sphere     1.0                   23.37 (±0.61)  
exp2       rastrigin  1.0                    1.82 (±0.24)  
           rosenbrock 1.0                214.89 (±387.12)  
           sphere     1.0                    0.45 (±0.14)  
exp3       rastrigin  1.0                   16.69 (±4.30)  
           rosenbrock 1.0                  78.06 (±48.66)  
           sphere     1.0                    0.02 (±0.00)  
exp4       rastrigin  1.0                  174.62 (±2.23)  
           rosenbrock 1.0   39,779,456.13 (±3,340,372.07)  
           sphere     1.0                  143.71 (±8.38)

In [6]:
print(final_data.to_csv(float_format='%.2f'))

experiment,function,freq,noNNReval_mof.csv
exp1,rastrigin,0.50,20.24 (±0.06)
exp1,rastrigin,1.00,21.90 (±0.47)
exp1,rastrigin,4.00,17.91 (±0.13)
exp1,rosenbrock,0.50,"71,719.74 (±4,327.15)"
exp1,rosenbrock,1.00,"83,986.73 (±3,445.84)"
exp1,rosenbrock,4.00,"85,830.59 (±1,274.26)"
exp1,sphere,0.50,19.27 (±1.30)
exp1,sphere,1.00,23.37 (±0.61)
exp1,sphere,4.00,18.42 (±0.09)
exp2,rastrigin,0.50,2.26 (±0.25)
exp2,rastrigin,1.00,1.82 (±0.24)
exp2,rastrigin,4.00,1.81 (±0.48)
exp2,rosenbrock,0.50,293.81 (±287.50)
exp2,rosenbrock,1.00,214.89 (±387.12)
exp2,rosenbrock,4.00,51.14 (±15.61)
exp2,sphere,0.50,0.49 (±0.14)
exp2,sphere,1.00,0.45 (±0.14)
exp2,sphere,4.00,0.39 (±0.34)
exp3,rastrigin,0.50,8.54 (±1.80)
exp3,rastrigin,1.00,16.69 (±4.30)
exp3,rastrigin,4.00,17.62 (±3.32)
exp3,rosenbrock,0.50,35.71 (±5.66)
exp3,rosenbrock,1.00,78.06 (±48.66)
exp3,rosenbrock,4.00,5.14 (±0.82)
exp3,sphere,0.50,0.04 (±0.00)
exp3,sphere,1.00,0.02 (±0.00)
exp3,sphere,4.00,0.01 (±0.00)
exp4,rastrigin,0.50,171.12 (±1

In [ ]:
!rm tmp.csv

In [ ]:
Func_names=['sphere','rastrigin','rosenbrock']
exp_names=['exp1','exp2','exp3','exp4']
frequency=1000
nn_window=5


In [ ]:
#Experiment = Enum('Experiment', 'exp1 exp2 exp3 exp4')
#Method = Enum('Methods', 'noNNRestart noNNReval NNnorm NNdrop')
#Mechname:Random, Worst, Closest
measure_name=['mof','sr','arr','nfe','nn_time']
index_measure=0
total=[]
#lables = ['noNNReval', 'NNnorm_Random', 'NNnorm_Worst','NNnorm_Closest','NNdrop_Random', 'NNdrop_Worst', 'NNdrop_Closest']
lables = ['noNNReval', 'NNnorm_Worst', 'NNdrop_Worst']

for experiment in exp_names:
    
    for func in Func_names:
       # path = Path(f'../../data/results/{experiment}/{func}')
        path_in = Path(f'../../data/cluster_results/{experiment}/{func}')
        path = path_in / f'freq{frequency}nn_w{nn_window}nn_p{3}'
        noNNReval= pd.read_csv(path/f'noNNReval_{measure_name[index_measure]}.csv').mean(0).values[0]

        #noNNRestart= pd.read_csv(path/'noNNRestart_mof.csv')

       # NNnorm_Random= pd.read_csv(path/f'NNnorm_Worst_{measure_name[index_measure]}.csv').mean(0).values[0]
       # NNdrop_Random = pd.read_csv(path/f'NNdrop_Worst_{measure_name[index_measure]}.csv').mean(0).values[0]


        NNnorm_Worst= pd.read_csv(path/f'NNnorm_Worst_{measure_name[index_measure]}.csv').mean(0).values[0]
        NNdrop_Worst = pd.read_csv(path/f'NNdrop_Worst_{measure_name[index_measure]}.csv').mean(0).values[0]


       # NNnorm_Closest= pd.read_csv(path/f'NNnorm_Closest_{measure_name[index_measure]}.csv').mean(0).values[0]
       # NNdrop_Closest = pd.read_csv(path/f'NNdrop_Closest_{measure_name[index_measure]}.csv').mean(0).values[0]
        
       # x=np.array([noNNReval.mof, NNnorm_Random.mof, NNnorm_Worst.mof, NNnorm_Closest.mof, NNdrop_Random.mof,NNdrop_Worst.mof,NNdrop_Closest.mof])
        #y=[noNNReval, NNnorm_Random,NNnorm_Worst,NNnorm_Closest,NNdrop_Random, NNdrop_Worst, NNdrop_Closest]
        y=[noNNReval, NNnorm_Worst,NNdrop_Worst]

        total.append(y)
        #print(total)

    #,  index=[Func_names,Func_names]

#df=pd.DataFrame(total, columns=lables,index=['sphere','rastrigin','rosenbrock','sphere','rastrigin','rosenbrock'])
df=pd.DataFrame(total, columns=lables,index=['sphere','rastrigin','rosenbrock','sphere','rastrigin','rosenbrock','sphere','rastrigin','rosenbrock', 'sphere','rastrigin','rosenbrock'])

#pd.options.display.float_format = '{:,.2f}'.format


print(pd.DataFrame.to_csv(df, float_format='%.2f'))


#pd.DataFrame.to_latex(df, multirow=True)    #, float_format="%%.2f" ,, , longtable=True,  bold_rows=True,


In [ ]:
pd.read_csv('tmp.csv')

In [ ]:
df

In [ ]:
#report nn_time
measure_name=['mof','sr','arr','nfe','nn_time']
index_measure=4
total=[]
lables = [ 'NNnorm_Random', 'NNnorm_Worst','NNnorm_Closest','NNdrop_Random', 'NNdrop_Worst', 'NNdrop_Closest']

for experiment in exp_names:
    
    for func in Func_names:
        path = Path(f'../../data/results/{experiment}/{func}')

        NNnorm_Random= pd.read_csv(path/f'NNnorm_Worst_{measure_name[index_measure]}.csv').mean(0).values[0]
        NNdrop_Random = pd.read_csv(path/f'NNdrop_Worst_{measure_name[index_measure]}.csv').mean(0).values[0]


        NNnorm_Worst= pd.read_csv(path/f'NNnorm_Worst_{measure_name[index_measure]}.csv').mean(0).values[0]
        NNdrop_Worst = pd.read_csv(path/f'NNdrop_Worst_{measure_name[index_measure]}.csv').mean(0).values[0]

        NNnorm_Closest= pd.read_csv(path/f'NNnorm_Closest_{measure_name[index_measure]}.csv').mean(0).values[0]
        NNdrop_Closest = pd.read_csv(path/f'NNdrop_Closest_{measure_name[index_measure]}.csv').mean(0).values[0]
        
       # x=np.array([noNNReval.mof, NNnorm_Random.mof, NNnorm_Worst.mof, NNnorm_Closest.mof, NNdrop_Random.mof,NNdrop_Worst.mof,NNdrop_Closest.mof])
        y=[ NNnorm_Random,NNnorm_Worst,NNnorm_Closest,NNdrop_Random, NNdrop_Worst, NNdrop_Closest]
        total.append(y)
        #print(total)


df=pd.DataFrame(total, columns=lables,index=['sphere','rastrigin','rosenbrock','sphere','rastrigin','rosenbrock'])



print(pd.DataFrame.to_csv(df, float_format='%.2f'))
#df
